# Anàlisi de sentiment

En aquesta secció es realitza un anàlisi de sentiment sobre els tuits, com exemple d'utilització de models de llenguatge pre-entrenats. En aquest cas, s'utilitza el model [DistilBERT](https://arxiv.org/abs/1910.01108) per a l'anàlisi de sentiment. Aquest model és una versió més lleugera del model [BERT](https://arxiv.org/abs/1810.04805), que és un model de llenguatge pre-entrenat que s'ha utilitzat amb molt bons resultats en diferents tasques de processament de llenguatge natural, com ara [anàlisi de sentiment](https://arxiv.org/abs/1905.05583), [classificació de text](https://arxiv.org/abs/1904.09077) o [extracció d'informació](https://arxiv.org/abs/1906.05237).

 En aquest cas, s'utilitza el model pre-entrenat per a l'anàlisi de sentiment en anglès.

## Càrrega del `dataset`

Utilitzarem la llibreria [datasets](https://huggingface.co/docs/datasets/) per a carregar el `dataset` de tuits. Aquesta llibreria permet carregar `datasets` de diferents fonts, com ara [Hugging Face Hub](https://huggingface.co/datasets), [Amazon AWS](https://docs.aws.amazon.com/es_es/marketplace/latest/userguide/datasets.html) o [Google Cloud](https://cloud.google.com/ai-platform/training/docs/datasets). En aquest cas, carregarem el `dataset` de tuits des de [Hugging Face Hub](https://huggingface.co/datasets/dair-ai/emotion).


In [1]:
# Instal·lem les lliberies que anem a utilitzar

%pip install transformers datasets evaluate accelerate keras scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [2]:
import datasets

# Carreguem el dataset
dataset = datasets.load_dataset('dair-ai/emotion')

# Mostrem les dades d'exemple

dataset['train'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0}

Podem veure que cada registre del `dataset` conté el text del tuit i el sentiment associat. En aquest cas, el sentiment està codificat amb un enter entre 0 i 5, on 0 correspon a sadness, 1 a joy, 2 a love, 3 a anger, 4 a fear i 5 a surprise.

## Preparació del `dataset`

En aquest cas, el `dataset` ja està dividit en conjunts d'entrenament, test i validació. El següent pas és preparar el `dataset` per a l'entrenament del model. En aquest cas, el model que utilitzarem és el model [BERT](https://arxiv.org/abs/1810.04805). Aquest model requereix que el text estigui tokenitzat i que els tokens estiguin codificats amb els seus identificadors numèrics corresponents. Per a això, utilitzarem un tokenitzador de DistilBERT pre-entrenat.

In [3]:
# importem el tokenitzador de DistilBERT
from transformers import AutoTokenizer

# Carreguem el tokenitzador
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Mostrem un exemple de tokenització
tokenizer('FC Barcelona is fucked this year')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 4429, 7623, 2003, 21746, 2023, 2095, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [4]:
# Definim una funció per pre-processar el text.
# Truncarem els textos per assegurar-nos que no superen el tamnay màxim d'entrada de DistilBert

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

Per aplicar la tokenització, utilitzarem la funció `map` de `datasets`. Aquesta funció permet aplicar una funció a cada registre del `dataset`. En aquest cas, la funció que aplicarem és la funció `tokenize` que hem definit anteriorment. Utilitzarem també `batched=True` per a indicar que la funció s'aplicarà a tot el `dataset` en blocs.

In [5]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

A continuació utilitzarem `DataCollatorWithPadding` per a crear un `DataCollator`, que és l'objecte que s'encarrega de crear els tensors d'entrada del model. En aquest cas, utilitzarem `DataCollatorWithPadding` per a crear tensors de mida fixa, afegint padding als tensors més curts. Això és necessari per a poder processar els tensors en blocs.

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## Avaluació

Per a avaluar el model, hem de carregar la métrica que utilitzarem per a l'avaluació. En aquest cas, utilitzarem la métrica `accuracy` del mòdul `evaluate` de HuggingFace.

També definirem una funció per a calcular les mètriques del model. Aquesta funció serà utilitzada per a avaluar el model després de cada epoch.

In [7]:
import evaluate

accuracy = evaluate.load('accuracy')

In [8]:
# Definim una funció per a calcular la precisió del model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Definició de les etiquetes

Abans d'entrenar el model hem de crear un diccionari que tradueixi els identificadors numèrics del sentiment a les seves etiquetes corresponents i a l'inrevés.

In [9]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

## Fine tuning del model

El procés de fine tuning del model és el d'entrenar el model amb el nostre `dataset`. Aixó permet que el model s'adapte millor a les nostres dades i millori el seu rendiment.

Hem de definir la funció d'optimització, el tamany dels blocs i el nombre d'epochs.

In [10]:
BATCH_SIZE = 16
NUM_EPOCHS = 5

In [11]:
from transformers import create_optimizer

# Definim la funció d'optimització
batches_per_epoch = len(dades_tokenitzades['train']) // BATCH_SIZE
total_steps = batches_per_epoch * NUM_EPOCHS
total_train_steps = int(total_steps * NUM_EPOCHS)
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=total_train_steps,
    num_warmup_steps=0,
)

Ara ja podem carregar el model pre-entrenat i fer el fine tuning. Utilitzarem `TFAutoModelForSequenceClassification` i li agregarem les etiquetes que hem definit anteriorment.

In [12]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

HuggingFace proporciona els models en format `pytorch` per defecte. Per a utilitzar-los amb `tensorflow` hem d'utilitzar la funció `prepare_tf_dataset` del model.

In [13]:
tf_train_dataset = model.prepare_tf_dataset(
    dades_tokenitzades['train'],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    shuffle=True
)

tf_val_dataset = model.prepare_tf_dataset(
    dades_tokenitzades['validation'],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    shuffle=False
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Per utilitzar la funció d'avaluació que hem definit anteriorment, hem d'utilitzar un `KerasMetricCallback` de HuggingFace. Aquest callback farà que la funció `compute_metrics` s'executi després de cada epoch.

In [14]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_dataset)

Ara ja podem compilar i entrenar el model, utilitzant el `optimizer` i el callback que hem definit anteriorment.

In [15]:
model.compile(optimizer=optimizer)
model.fit(tf_train_dataset, epochs=NUM_EPOCHS, validation_data=tf_val_dataset, callbacks=[metric_callback])

Epoch 1/5
1000/1000 [==============================] - 142s 118ms/step - loss: 0.4693 - val_loss: 0.1866 - accuracy: 0.9370
Epoch 2/5
1000/1000 [==============================] - 104s 104ms/step - loss: 0.1379 - val_loss: 0.1467 - accuracy: 0.9380
Epoch 3/5
1000/1000 [==============================] - 104s 104ms/step - loss: 0.0999 - val_loss: 0.1373 - accuracy: 0.9370
Epoch 4/5
1000/1000 [==============================] - 104s 104ms/step - loss: 0.0892 - val_loss: 0.1488 - accuracy: 0.9380
Epoch 5/5
1000/1000 [==============================] - 103s 103ms/step - loss: 0.0735 - val_loss: 0.1757 - accuracy: 0.9295


## Inferència

Per a fer inferència amb el model, crearem un pipeline de HuggingFace. Aquest pipeline utilitzarà el model i el tokenitzador que hem importat anteriorment.

A continuació, utilitzarem el pipeline per a fer inferència amb un text d'exemple.

In [24]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("FC Barcelona is fucked this year"))
print(classifier("I'm not going to watch soccer again"))
print(classifier("Real Madrid is not going to treble this year. I'm relieved!"))

[{'label': 'ANGER', 'score': 0.9984560012817383}]
[{'label': 'SADNESS', 'score': 0.6338914632797241}]
[{'label': 'JOY', 'score': 0.9984081387519836}]
